In [25]:
import sqlite3

In [26]:
sqlite3.version

'2.6.0'

In [27]:
sqlite3.sqlite_version

'3.33.0'

In [28]:
con = sqlite3.connect("E:\\Temp\\kospi.db")
type(con)

sqlite3.Connection

In [29]:
cursor = con.cursor()

In [30]:
cursor.execute("CREATE TABLE kakao(Date text, Open int, High int, Low int, Closing int, Volumn int)")

OperationalError: table kakao already exists

In [31]:
cursor.execute("INSERT INTO kakao VALUES('16.06.03', 97000, 98600, 96900, 98000, 32145)")
cursor.execute("INSERT INTO kakao VALUES('16.06.02', 99000, 99300, 96300, 97500, 556790)")

execute 메서드를 사용해 SQL 구문을 실행할 때 한 가지 주의할 점이 있습니다. SQL 구문을 실행하는 코드를 살펴보면 SQL 구문 전체를 큰따옴표를 사용해서 문자열로 만들었습니다. 따라서 '16.06.03'과 같이 SQL 구문에 문자열이 있는 경우 작은따옴표를 사용해야 합니다.

In [32]:
con.commit()
con.close()

In [33]:
con = sqlite3.connect("E:\\Temp\\kospi.db")
cursor = con.cursor()

In [34]:
cursor.execute("SELECT * FROM kakao")

In [35]:
cursor.fetchone()

('16.06.03', 97000, 98600, 96900, 98000, 32145)

In [36]:
cursor.fetchone()

('16.06.02', 99000, 99300, 96300, 97500, 556790)

In [37]:
cursor.execute("SELECT * FROM kakao")

In [38]:
cursor.fetchall()

[('16.06.03', 97000, 98600, 96900, 98000, 32145),
 ('16.06.02', 99000, 99300, 96300, 97500, 556790),
 ('16.06.03', 97000, 98600, 96900, 98000, 32145),
 ('16.06.02', 99000, 99300, 96300, 97500, 556790)]

In [39]:
cursor.execute("SELECT * FROM kakao")
kakao = cursor.fetchall()

In [40]:
kakao[0][0]

'16.06.03'

In [41]:
kakao[0][1]

97000

## Save DateFrame Data to SQLite DB

In [42]:
from pandas import Series, DataFrame
raw_data = {'col0': [1, 2, 3, 4], 'col1': [10, 20, 30, 40], 'col2':[100, 200, 300, 500]}
df = DataFrame(raw_data)
df

,col0,col1,col2
0,1,10,100
1,2,20,200
2,3,30,300
3,4,40,500


In [43]:
con = sqlite3.connect("E:\\Temp\\kospi.db")
df.to_sql('test', con, if_exists='replace')

```python
DataFrame.to_sql(name, con, flavor='sqlite', schema=None, if_exists='fail', index=True, index_label=None, chunksize=None, dtype=None)
```
to_sql 메서드에는 여러 파라미터가 있지만 대부분 기본값을 가지고 있어 따로 지정할 필요가 없는 경우가 많습니다. 다만 DataFrame 객체에 많은 로우가 있어 이를 데이터베이스 한 번에 쓰는 경우 패킷 크기 제약으로 에러가 발생할 수 있습니다. 이 경우 다음과 같이 한 번에 데이터베이스로 저장될 로우의 개수를 지정하면 됩니다.
```python
df.to_sql('test', con, chunksize=1000)
```

## Load DataFrame Data from SQLite DB

In [44]:
import pandas as pd
from pandas import Series, DataFrame

In [45]:
df = pd.read_sql("SELECT * FROM kakao", con, index_col=None)
df

,Date,Open,High,Low,Closing,Volumn
0,16.06.03,97000,98600,96900,98000,32145
1,16.06.02,99000,99300,96300,97500,556790
2,16.06.03,97000,98600,96900,98000,32145
3,16.06.02,99000,99300,96300,97500,556790


In [46]:
df = pd.read_sql("SELECT * FROM test", con, index_col='index')
df

,col0,col1,col2
index,,,
0,1,10,100
1,2,20,200
2,3,30,300
3,4,40,500


## Save Stock Quotes in DB

In [47]:
import pandas as pd
import pandas_datareader.data as web
import datetime
import sqlite3

In [48]:
start = datetime.datetime(2010, 1, 1)
end = datetime.datetime(2016, 6, 12)
df = web.DataReader("078930.KS", "yahoo", start, end)

In [49]:
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-01-04,34050.0,33500.0,33500.0,33750.0,237735.0,26427.386719
2010-01-05,34500.0,33600.0,33950.0,33900.0,440485.0,26544.845703
2010-01-06,34900.0,33900.0,33900.0,34250.0,534581.0,26818.904297
2010-01-07,35100.0,34300.0,34350.0,34600.0,543769.0,27092.962891
2010-01-08,35100.0,34200.0,34700.0,34450.0,553288.0,26975.513672


In [50]:
con = sqlite3.connect("E:\\Temp\\kospi.db")
df.to_sql('078930', con)

E:\anaconda32bit\anaconda3\envs\MyKiwoom\lib\site-packages\pandas\core\generic.py:2602: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [53]:
readed_df = pd.read_sql("SELECT * FROM '078930'", con, index_col='Date')